In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('leica_subreddit_posts.csv')

In [3]:
df

,post_id,title,post_body,score,upvote_ratio,url,author,created_utc,created_date,num_comments,post_flair,is_moderator_post,is_edited,awards,gilded,crosspost_count
0,40vyl9,Posting Guidelines for Imagery/Photography,"For those who are new to the subreddit, or may...",103,0.98,https://www.reddit.com/r/Leica/comments/40vyl9...,SnowHawkMike,1.452745e+09,2016-01-14 04:12:06,0,NaN,False,False,0,0,0
1,1f2y7wq,What I'd like you to know about flair-- Please...,Most of my time is spent answering questions a...,10,0.86,https://www.reddit.com/r/Leica/comments/1f2y7w...,dethswatch,1.724809e+09,2024-08-28 01:36:23,10,I TYPED MY OWN FLAIR,False,False,0,0,0
2,1g3w684,My entire collection of 14 leica lenses fit co...,I use spent film cartridges to prop up smaller...,48,0.94,https://www.reddit.com/gallery/1g3w684,OnePhotog,1.728955e+09,2024-10-15 01:19:49,9,NaN,False,False,0,0,0
3,1g3gcwv,Finally pulled the trigger my first Leica!,NaN,320,0.81,https://i.redd.it/rs8a587b1joc1.jpeg,passionecstasy,1.728914e+09,2024-10-14 13:56:02,32,NaN,False,False,0,0,0
4,1g3p683,Text Peeled Off - 2 Week Old Leica MP,"I recently got my first Leica, a brand new bla...",57,0.93,https://www.reddit.com/gallery/1g3p683,currynesque30,1.728936e+09,2024-10-14 19:59:26,54,NaN,False,False,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,1f10hy8,Diffusion Filter: Tiffen vs Urth,"Hey fam, here in Germany its not that easy to ...",2,1.00,https://www.reddit.com/r/Leica/comments/1f10hy...,thenickfo,1.724604e+09,2024-08-25 16:33:40,13,NaN,False,False,0,0,0
994,1f0rol9,Leaving M film cameras with shutter charged. I...,"Hi, all. Hope you are having a great summer. H...",5,0.86,https://www.reddit.com/r/Leica/comments/1f0rol...,OPinjapan,1.724575e+09,2024-08-25 08:29:14,6,NaN,False,False,0,0,0
995,1f0tv7y,Has anyone done IR photography with the Leica ...,I took my CL out yesterday for some IR photogr...,3,1.00,https://www.reddit.com/r/Leica/comments/1f0tv7...,tominabox1,1.724584e+09,2024-08-25 11:06:57,0,NaN,False,False,0,0,0
996,1ezy5b8,chatGTP's attempt to roast my Leica,NaN,491,0.94,https://i.redd.it/eru5317evjkd1.png,Educatenrepeat,1.724478e+09,2024-08-24 05:45:11,57,NaN,False,False,0,0,0


In [4]:
import praw
import time
import pandas as pd
from datetime import datetime

client_id = 'NM5z8s-orOjTgXYl41n67A'
client_secret = '-Rdrus6GAHHRcm2D5xPPA9zJrr0C9A'
user_agent = 'reddit_scraper'

# Set up PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

# Choose the subreddit 'Leica'
subreddit = reddit.subreddit('Leica')

# Function to convert Unix timestamp to a human-readable date
def get_post_date(created_utc):
    return datetime.utcfromtimestamp(created_utc).strftime('%Y-%m-%d %H:%M:%S')

# Function to scrape comments from a submission
def get_comments(submission):
    submission.comments.replace_more(limit=None)  # Replace "MoreComments" objects to get all comments
    comments_data = []
    
    for comment in submission.comments.list():
        comments_data.append({
            'comment_id': comment.id,
            'comment_author': str(comment.author) if comment.author else '[deleted]',  # Handle deleted comments
            'comment_body': comment.body,
            'comment_score': comment.score,
            'comment_created_utc': comment.created_utc,
            'comment_created_date': get_post_date(comment.created_utc),
            'comment_upvote_ratio': comment.upvote_ratio if hasattr(comment, 'upvote_ratio') else None,
            'comment_awards': comment.total_awards_received
        })
    
    return comments_data

# Scrape posts and comments, storing results in a DataFrame
def scrape_leica_subreddit(limit=5):
    data = []  # List to store each post and its details
    
    for submission in subreddit.hot(limit=limit):  # Use subreddit.new(), .top(), or others as needed
        print(f"Scraping submission: {submission.title}")
        
        # Handle case where author is deleted
        author = submission.author
        if author:
            author_name = str(author)
            # author_comment_karma = author.comment_karma
            # author_link_karma = author.link_karma
        else:
            author_name = '[deleted]'
            author_comment_karma = None
            author_link_karma = None

        # Store post details as a dictionary
        post_data = {
            'post_id': submission.id,
            'title': submission.title,
            'post_body': submission.selftext,  # If the post is a text post
            'score': submission.score,
            'upvote_ratio': submission.upvote_ratio,
            'url': submission.url,
            'author': author_name,
            # 'author_comment_karma': author_comment_karma,
            # 'author_link_karma': author_link_karma,
            'created_utc': submission.created_utc,
            'created_date': get_post_date(submission.created_utc),  # Convert to human-readable date
            'num_comments': submission.num_comments,
            'post_flair': submission.link_flair_text,  # Extract post flair
            'is_moderator_post': submission.distinguished == 'moderator',  # Check if post is from a moderator
            'is_edited': submission.edited if submission.edited else False,  # Check if the post was edited
            'awards': submission.total_awards_received,  # Total number of awards
            'gilded': submission.gilded,  # Number of times the post was gilded
            'crosspost_count': len(submission.crosspost_parent_list) if hasattr(submission, 'crosspost_parent_list') else 0,  # Check if crossposted
            'comments': get_comments(submission)  # Get all comments as a list of dictionaries
        }
        data.append(post_data)
    
    # Convert list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Run the scraper and store the results in a DataFrame (limit=100 posts as an example)
df = scrape_leica_subreddit()

# Expand the comments into a separate DataFrame if needed
comments_data = []
for index, row in df.iterrows():
    for comment in row['comments']:
        comment['post_id'] = row['post_id']  # Keep reference to the parent post
        comments_data.append(comment)

comments_df = pd.DataFrame(comments_data)

# Drop the 'comments' column from the posts DataFrame, since it's now stored separately
# df = df.drop(columns=['comments'])

# # Display the DataFrames
# print("Posts Data:")
# print(df.head())
# print("\nComments Data:")
# print(comments_df.head())

# # Optional: Save both DataFrames to CSV files
# df.to_csv('leica_subreddit_posts.csv', index=False)
# comments_df.to_csv('leica_subreddit_comments.csv', index=False)


Scraping submission: Posting Guidelines for Imagery/Photography
Scraping submission: What I'd like you to know about flair-- Please check it out if you care about flair
Scraping submission: Had some custom work done to my M11. 
Scraping submission: Debate. Is he right?
Scraping submission: Just got it back from a CLA, & ready to shoot!


In [5]:
df

,post_id,title,post_body,score,upvote_ratio,url,author,created_utc,created_date,num_comments,post_flair,is_moderator_post,is_edited,awards,gilded,crosspost_count,comments
0,40vyl9,Posting Guidelines for Imagery/Photography,"For those who are new to the subreddit, or may...",102,0.98,https://www.reddit.com/r/Leica/comments/40vyl9...,SnowHawkMike,1.452745e+09,2016-01-14 04:12:06,0,None,False,False,0,0,0,[]
1,1f2y7wq,What I'd like you to know about flair-- Please...,Most of my time is spent answering questions a...,10,0.86,https://www.reddit.com/r/Leica/comments/1f2y7w...,dethswatch,1.724809e+09,2024-08-28 01:36:23,10,I TYPED MY OWN FLAIR,False,False,0,0,0,"[{'comment_id': 'lkab3li', 'comment_author': '..."
2,1g4oc8y,Had some custom work done to my M11.,Had some custom work at the Leica HQ during a ...,52,1.00,https://i.redd.it/7hawyk9kx0vd1.jpeg,Madvillun85,1.729043e+09,2024-10-16 01:48:58,13,None,False,False,0,0,0,"[{'comment_id': 'ls4zkoy', 'comment_author': '..."
3,1g4jf59,Debate. Is he right?,,86,0.85,https://i.redd.it/zbcw2uasqzud1.jpeg,Brekkeks,1.729029e+09,2024-10-15 21:49:12,66,None,False,False,0,0,0,"[{'comment_id': 'ls3ygya', 'comment_author': '..."
4,1g4fsud,"Just got it back from a CLA, & ready to shoot!",,88,0.97,https://i.redd.it/u4yc3an6zyud1.jpeg,OutrageousPhysicaliy,1.729020e+09,2024-10-15 19:14:30,18,None,False,False,0,0,0,"[{'comment_id': 'ls36mh6', 'comment_author': '..."


In [7]:
df['comments'][1]

[{'comment_id': 'lkab3li',
  'comment_author': 'DigitalHeMan',
  'comment_body': 'I came into this thread expecting to hear about lens flare. But thank you for explaining this equally important topic 😅',
  'comment_score': 5,
  'comment_created_utc': 1724815389.0,
  'comment_created_date': '2024-08-28 03:23:09',
  'comment_upvote_ratio': None,
  'comment_awards': 0,
  'post_id': '1f2y7wq'},
 {'comment_id': 'lkacir6',
  'comment_author': 'HorkusSnorkus',
  'comment_body': 'Thanks for doing this.  You are hereby promoted the Hero And Patriot 1st Class With Summicron Clusters',
  'comment_score': 4,
  'comment_created_utc': 1724816015.0,
  'comment_created_date': '2024-08-28 03:33:35',
  'comment_upvote_ratio': None,
  'comment_awards': 0,
  'post_id': '1f2y7wq'},
 {'comment_id': 'lkare6t',
  'comment_author': 'turo9992000',
  'comment_body': "I'm one of the ones that messaged about flair last year, sorry. \n\nIt is important that people know what cameras I have though.",
  'comment_score